In [30]:
import numpy as np
import pandas as pd
import pymongo
import os
from glob import glob
import pdb

In [31]:
import json
jsonPath = os.path.join('/storage/goship/same_expocode_json/')
files = glob(os.path.join(jsonPath, '**/*.json'))
print(jsonPath)
print(len(files))

/storage/goship/same_expocode_json/
350


In [29]:
def make_doc(filename):
    with open(filename) as f:
        doc = json.load(f)
    return doc

def create_collection(dbName, collectionName):
    dbUrl = 'mongodb://localhost:27017/'
    client = pymongo.MongoClient(dbUrl)
    db = client[dbName]
    coll = db[collectionName]
    coll = init_profiles_collection(coll)
    return coll

def init_profiles_collection(coll):
    coll.create_index([('date', pymongo.DESCENDING)])
    coll.create_index([('lat', pymongo.DESCENDING)])
    coll.create_index([('lon', pymongo.DESCENDING)])
    coll.create_index([('geoLocation', pymongo.DESCENDING)])
    return coll

dbName = 'goship'
collName = 'profiles'
coll = create_collection(dbName, collName)
coll.drop()


for file in files:
    doc = make_doc(file)
    coll.insert_one(doc)

In [75]:
coll.find_one( {'expocode': '33KMGP402_1'} )

{'_id': ObjectId('5f7bbba5afc6ec18cbc70aff'),
 'whp_name': 'CTDSAL',
 'whp_unit': 'PSS-78',
 'standard_name': 'sea_water_practical_salinity',
 'units': '1',
 'reference_scale': 'PSS-78',
 'ancillary_variables': 'ctd_salinity_qc',
 'name': 'ctd_salinity',
 'latitude': 7.005,
 'longitude': 165.075,
 'time': '2002-07-06T12:06:00',
 'expocode': '33KMGP402_1',
 'station': '2',
 'cast': 1,
 'sample': ['13', '12', '11', '10', '9', '8', '7', '6', '5', '4', '3', '1'],
 'measurements': [{'data': 33.90599822998047, 'pres': 3.2},
  {'data': 33.90700149536133, 'pres': 9.9},
  {'data': 33.92660140991211, 'pres': 24.5},
  {'data': 33.93960189819336, 'pres': 39.6},
  {'data': 34.27519989013672, 'pres': 60.0},
  {'data': 34.880699157714844, 'pres': 100.3},
  {'data': 34.55080032348633, 'pres': 150.5},
  {'data': 34.605098724365234, 'pres': 200.0},
  {'data': 34.625099182128906, 'pres': 400.8},
  {'data': 34.56570053100586, 'pres': 600.2},
  {'data': 34.546600341796875, 'pres': 799.0},
  {'data': 34.561